In [1]:
%load_ext autoreload
%autoreload 2

## import 

In [2]:
import numpy as np
import pandas as pd
from kite_trade import *
import os
# import ta

import talib
import datetime
import pyarrow.parquet as pq
import pyarrow as pa

import multiprocessing
import time
from tqdm import tqdm
import util_fun as uf

from pandas.tseries.offsets import MonthEnd
from datetime import timedelta

import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import mplfinance as mpf


from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA


In [3]:
enctoken = "riRT1ez1qcMCdYvosPUjYNLUDoSE7m80wjUkxLoJBWZtZmn1uBmBnG38JkbDgyEI/JpFUieYzICN4SoTxK8hMZUPS8eLHEfTAwy8AU0nKtG6NOBXApsHEA=="
kite = KiteApp(enctoken=enctoken)

In [4]:
timerframe_list = [
"day",
"minute",
 "3minute",
 "5minute",
 "10minute",
 "15minute",
 "30minute",
 "60minute",]

# instument in nse
inst = pd.DataFrame(kite.instruments("NSE"))
inst_filter = inst.query('(name != "")').copy()
inst_filter.rename(columns = {"tradingsymbol":'Symbol'},inplace = True)
# inst_filter.query("Symbol. 'NIFTY'")
inst_filter[inst_filter.Symbol.str.contains('HDFCBANK')]


,instrument_token,exchange_token,Symbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
3134,341249,1333,HDFCBANK,HDFC BANK,0.0,None,0.0,0.05,1,EQ,NSE,NSE


## stretegy 1 OD

In [ ]:
# buy: 1st candle breaks above the PDH 
# 	target-> next pivot or 2R
# 	stop loss -> 1st candle low
# sell: reverse of above
# cond: 1. first candle should be not to big
# 	2. avoid to much gap up and down trade

In [68]:
start_dt = '2022-01-01'
end_dt = '2023-03-12'
time_frame = '5minute'
instument =256265 #256265(nifty) # 260105(bank) #341249(hdfc)
df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/14 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/14 [00:00<?, ?it/s]

In [70]:
df_day_fetch['prev_high']  = df_day_fetch.High.shift(1)
df_day_fetch['prev_low']  = df_day_fetch.Low.shift(1)

df_day_fetch.reset_index(inplace=True)
df_day_fetch['date_only'] = pd.to_datetime(df_day_fetch.Date.dt.date)
df_5min.reset_index(inplace=True)
df_5min['date_only'] =pd.to_datetime(df_5min.Date.dt.date)
df_5min_1 = df_5min.merge(df_day_fetch, how = 'left', on = 'date_only', suffixes=('_min','_day'))
df_5min_1.dropna(inplace = True)


In [83]:
# (
#     df_5min_1[
#         (df_5min_1.Date_min.dt.time.isin([pd.to_datetime('09:15:00').time(),pd.to_datetime('09:15:00').time(),pd.to_datetime('09:15:00').time()]) )
#         & (df_5min_1.Close_min >= df_5min_1.prev_high)
#         & (df_5min_1.Open_min < df_5min_1.Close_min)
        
#     ]
# )

In [84]:
# (
#     df_5min_1[
#         (df_5min_1.Date_min.dt.time.isin([pd.to_datetime('09:15:00').time(),pd.to_datetime('09:20:00').time(),pd.to_datetime('09:25:00').time()]) )
#         & (df_5min_1.Close_min <= df_5min_1.prev_low)
#         & (df_5min_1.Open_min > df_5min_1.Close_min)
#         & ((df_5min_1.prev_low - df_5min_1.Open_min)  < 10)
#     ]
# )

In [90]:
(
    df_5min_1[
        (df_5min_1.Date_min.dt.time.isin([pd.to_datetime('09:15:00').time(),pd.to_datetime('09:20:00').time(),pd.to_datetime('09:25:00').time()]) )
        & (df_5min_1.Close_min >= df_5min_1.prev_low)
        & (df_5min_1.Open_min < df_5min_1.prev_low)
        
    ]
)

,Date_min,Open_min,High_min,Low_min,Close_min,Volume_min,date_only,Date_day,Open_day,High_day,Low_day,Close_day,Volume_day,prev_high,prev_low
226,2022-01-06 09:20:00+05:30,17747.00,17784.25,17738.90,17762.55,0,2022-01-06,2022-01-06 00:00:00+05:30,17768.50,17797.95,17655.55,17745.90,0,17944.70,17748.84
676,2022-01-14 09:20:00+05:30,18132.70,18181.20,18129.10,18181.00,0,2022-01-14,2022-01-14 00:00:00+05:30,18185.00,18286.95,18119.65,18255.75,0,18272.25,18163.80
1801,2022-02-04 09:20:00+05:30,17501.75,17551.15,17480.90,17551.15,0,2022-02-04,2022-02-04 00:00:00+05:30,17590.20,17617.80,17462.55,17516.30,0,17781.15,17511.15
1875,2022-02-07 09:15:00+05:30,17456.30,17535.95,17437.00,17528.05,0,2022-02-07,2022-02-07 00:00:00+05:30,17456.30,17536.75,17119.40,17213.59,0,17617.80,17462.55
4202,2022-03-24 09:25:00+05:30,17176.20,17201.50,17166.65,17200.15,0,2022-03-24,2022-03-24 00:00:00+05:30,17094.95,17291.75,17091.15,17222.75,0,17442.40,17199.60
5701,2022-04-22 09:20:00+05:30,17210.50,17219.20,17201.40,17215.85,0,2022-04-22,2022-04-22 00:00:00+05:30,17242.75,17315.30,17149.20,17171.95,0,17414.70,17215.50
8252,2022-06-09 09:25:00+05:30,16289.35,16319.40,16287.45,16314.05,0,2022-06-09,2022-06-09 00:00:00+05:30,16263.85,16492.80,16243.85,16478.10,0,16514.30,16293.35
8475,2022-06-14 09:15:00+05:30,15674.25,15765.15,15674.25,15738.10,0,2022-06-14,2022-06-14 00:00:00+05:30,15674.25,15858.00,15659.45,15732.10,0,15886.15,15684.00
9302,2022-06-29 09:25:00+05:30,15708.00,15721.75,15700.80,15717.40,0,2022-06-29,2022-06-29 00:00:00+05:30,15701.70,15861.60,15687.80,15799.10,0,15892.10,15710.15
9975,2022-07-11 09:15:00+05:30,16136.15,16165.30,16120.75,16158.60,0,2022-07-11,2022-07-11 00:00:00+05:30,16136.15,16248.55,16115.50,16216.00,0,16275.50,16157.90


In [67]:
# df_5min_1['buy_cond'] = df_5min_1